# Earthquake data from USGS

This notebooks live data from the USGS.

In [6]:
%pip install folium
%pip install ipywidgets



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import folium
from folium.plugins import MarkerCluster
import requests
import webbrowser
from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
import threading
from ipywidgets import Button, HBox
import IPython.display as display

# Get USGS earthquake data (last 30 days, M2.5+ earthquakes)
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson"
response = requests.get(url)
data = response.json()

# Create a map centered on the US
m = folium.Map(location=[39.50, -98.35], zoom_start=4)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add earthquake markers to the cluster
for feature in data['features']:
    coords = feature['geometry']['coordinates']
    lat, lon = coords[1], coords[0]
    mag = feature['properties']['mag']
    place = feature['properties']['place']
    popup_text = f"<strong>Magnitude:</strong> {mag}<br><strong>Location:</strong> {place}"
    popup = folium.Popup(popup_text, max_width=300)
    folium.Marker([lat, lon], popup=popup).add_to(marker_cluster)

# Save the map as an index.html file
m.save("index.html")

# Serve the index.html file using a simple HTTP server
PORT = 8000
httpd = None

def start_server():
    global httpd
    handler = SimpleHTTPRequestHandler
    httpd = TCPServer(("", PORT), handler)
    print(f"Serving on port {PORT}. Open your browser and visit http://localhost:{PORT}")
    httpd.serve_forever()

# Start the server in a separate thread
server_thread = threading.Thread(target=start_server)
server_thread.start()

# Open the default web browser and load the page
webbrowser.open(f"http://localhost:{PORT}")

# Create a function to stop the server
def stop_server(button):
    global httpd
    if httpd:
        httpd.shutdown()
        server_thread.join()
        httpd.server_close()
        print("Server stopped.")
    else:
        print("Server not running.")

# Create a function to restart the server
def restart_server(button):
    global server_thread, httpd
    if httpd:
        stop_server(None)
    print("Restarting server...")
    server_thread = threading.Thread(target=start_server)
    server_thread.start()
    webbrowser.open(f"http://localhost:{PORT}")

# Create buttons and set their event handlers
stop_button = Button(description="Stop server")
stop_button.on_click(stop_server)

restart_button = Button(description="Restart server")
restart_button.on_click(restart_server)

# Display the buttons
button_container = HBox([stop_button, restart_button])
display.display(button_container)


Serving on port 8000. Open your browser and visit http://localhost:8000


127.0.0.1 - - [02/May/2023 16:43:18] "GET / HTTP/1.1" 200 -


In [7]:
stop_server


<function __main__.stop_server(button)>